# <center>Replay of round-robin study </center>

## Overview

Demonstration notebook using the data provided in the following study: "A round-robin approach provides a detailed
assessment of biomolecular small-angle scattering data reproducibility and yields consensus curves for
benchmarking" [1].


**References:**

[1] J. Trewhella et al., *Acta Cryst.* (2022), D**78**, 1315-1336. https://doi.org/10.1107/S2059798322009184

# Get started

To get started, import the necessary modules and set up the back-end for interactive plotting:

In [ ]:
from sas_helper import *
import numpy as np
import requests
%matplotlib notebook

And define dictionary with all systems and codes for PDB and SASBDB (see below for more information):

In [ ]:
codes = {}
codes['rnasea'] = ('7rsa.pdb', ['SASDPP4', 'SASDPU4', 'SASDP25'])
codes['lysozyme'] = ('2vb1.pdb', ['SASDPT4', 'SASDPV4', 'SASDPZ4'])
codes['xylanase'] = ('2dfc.pdb', ['SASDPS4', 'SASDPW4', 'SASDP35'])
codes['urate'] = ('3l8w.pdb', ['SASDPQ4', 'SASDPX4', 'SASDP45'])
codes['xylose'] = ('1mnz.pdb', ['SASDPR4', 'SASDPY4', 'SASDP55'])

And choose one of the systems above for the analysis, as well as the number of iterations and conformations to generate (see NOLB modelling below):

In [ ]:
system = 'rnasea'
ITERATIONS = 100
MODES = 15

# Load the PDB files

Crystal structures for the five proteins studied in [1] are available in the Protein Data Bank (PDB) [https://www.rcsb.org](https://www.rcsb.org) and their respective codes are:

- RNaseA: PDB entry = 7rsa

- Lysozyme: PDB entry = 2vb1 

- Xylanase: PDB entry = 2dfc

- Urate oxidase: PDB entry = 3l8w

- Xylose isomerase: PDB entry = 1mnz

They can be downloaded easily using the `download_pdb` function:

```python
download_pdb(link, pdb_file)
``` 

where link = `https://files.rcsb.org/download/entry-code` provides the reference to the PDB file in the database.

Let's load the reference structure of the five systems analysed in [1]:

In [ ]:
for key in codes.keys():
    url = 'https://files.rcsb.org/download/' + codes[key][0]
    download_pdb(url, key + '.pdb')

# Protein visualisation

Use the `show_pdb` command to visualize the loaded structures:

```python
show_pdb(pdb_file)
```

Use the interactive 3D visualization of the protein structure to explore the structure. You can rotate the protein, zoom in and out, and explore different parts of the model.

### Irina

The PDB models for urate and xylose do not look like those shown in Fig. 1 of the paper. For urate, it looks like chain A of uricase. Do several of these chains need to be combined to give the total structure? How to handle that?

The same for xylose. Note also that the code given in the paper (1mnz) retrieves the structure of glucose isomerase. But the entry has xylose isomerase as an entity. How many of those chains compose the real protein? Is it possible to get directly the full system?

In [ ]:
pdb_file = system + '.pdb'
show_pdb('xylose.pdb')

# Load the SANS/SAXS data

Consensus SANS (in both D2O and H2O) and SAXS data used in [1] are available in the SASBDB database. They can be downloaded manually or as shown below. The codes corresponding to each data set are given in [1] and have been defined before in the `codes` dictionary, where they are available as `codes[system key][1][#]`, where # = 0 (SAXS data), 1 (SANS data taken in D2O), or 2 (SANS data taken in H2O).

### Irina
The code below should be adapted and moved to sas_helper. py, so that it can be used in the notebook with a simple call similar to the one used for the pdb files, e.g. download_sasbdb_set.

In [ ]:
requests.packages.urllib3.disable_warnings()

API_BASE = 'https://www.sasbdb.org/rest-api/'
URL_BASE = 'https://www.sasbdb.org/media/intensities_files/'

# Check codes to use and download SANS/SAXS data curves locally
for key in codes.keys():
    for code in codes[key][1]:
        
        try:
            
            resp = requests.get(API_BASE + 'entry/summary/', params={'code': code},
                    headers={"Accept": "application/json"}, verify=False)
            if resp.status_code != 200:
                resp.raise_for_status()
            else:
                json_data =  resp.json() 
                print(json_data['code'], ':', json_data['experiment']['sample']['name'])
                
            url = URL_BASE + code + '.dat'
            resp = requests.get(url, allow_redirects=True, verify=False)
            file = code + '.dat'
            open(file, 'wb').write(resp.content)
    
        except requests.exceptions.HTTPError:
            resp.raise_for_status()

### Irina
Function in sas_helper.py to plot the curves from SASBDB?

In [ ]:
plt.figure()
comments = ['Sample', 'Parent', 'parent', 'template', 'creator', 'range', 'dat',  '#']

data = {}
for code, label in zip(codes[system][1], ['SAXS', 'SANS D2O', 'SANS H2O']):
    data[code] = np.loadtxt(code + '.dat', comments = comments)
    plt.plot(data[code][:,0], data[code][:,1], label=label)

plt.xlabel('Q/$\AA^{-1}$')
plt.ylabel('(IQ)')
plt.yscale('log')
plt.xlim((0,0.5))
plt.ylim((0.001,1.1))
plt.legend()
plt.show()   

# Calculation of SAXS Profiles

To calculate SAXS profiles, you can use either the `foxs` or `Pepsi-SAXS` software. There is no critical difference between them, except for the intensity scale. The  `saxs_profile` function will calculate the SAXS profile for a given PDB file. By default, the function uses the `foxs` software, so the call:

```python
saxs_profile(pdb_file)
```
is equivalent to 
```python
saxs_profile(pdb_file, core="foxs")
```

To calculate the SAXS profile using the `Pepsi-SAXS` software, use the following command:
```python
saxs_profile(pdb_file, core="pepsisaxs")
```

Or:
```python
saxs_profile(pdb_file, core="both")
```
to calculate the profile with both programs.

### Irina
I added a save in CSV format into saxs_profile and sans_profile in order to have all the calculated profiles (either with foxs or pepsisaxs) in the same format and facilitate the plot comparing calculation and experiment. Useful or not needed and one can do this as easily using the direct output of foxs and pepsisaxs?

I also think that the initial aspect of the plot should be lin/log.

In [ ]:
pdb_file = system + '.pdb'
saxs_profile(pdb_file, core="both")


If instead of a single file we provide a list of them, the profiles of all the systems in the list is calculated and compared in the output plot. For example we can compare the profiles of the five crystal structures in [1]:

In [ ]:
saxs_profile(['{}.pdb'.format(key) for key in codes.keys()], core='pepsisaxs')

# Calculation of SANS Profiles

SANS profiles are modeled in a similar way, but in this case we need to indicate the H/D levels of the protein and the buffer, so the functon `sans_profile` has some additional keywords:

```python
sans_profile(pdb_files, deut_level=[0], d2o_level=[0], exchange=[0])
```
- **deut_level** stands for Molecule deuteration. It represents the level of deuteration in the molecules under investigation. Deuteration refers to the replacement of hydrogen atoms with deuterium atoms. The **deut_level** value should be in the range from 0 to 1.
- **d2o_level** stands for Buffer deuteration. It represents the level of deuteration in the solvent or buffer used in the experiment. The **d2o_level** value should be in the range from 0 to 1.
- **exchange** stands for Exchange rate. It represents the rate at which hydrogen atoms in the molecule exchange with deuterium atoms. Higher exchange rates indicate faster exchange between hydrogen and deuterium atoms. The **exchange** value should be in the range from 0 to 1.

Default values are: deut_level=0, d2o_level=0, and exchange=0.

To explore how different parameter values affect the SANS profiles, you can provide a list of different numbers for each parameter. The `sans_profile` function will calculate all the combinations of those numbers, for example:

In [ ]:
pdb_file = system + '.pdb'
sans_profile(pdb_file, deut_level=[0,1], d2o_level=[0, 1], exchange=0)

# Calculated vs experimental profiles

Before doing any fitting, let's simply compare how close the profiles of the reference structure are to the experimental data.

### Irina

Again, useful to add to sas_helper.py?

In [ ]:
plt.figure()
data = {}

for code, label in zip(codes[system][1], ['SAXS', 'SANS D2O', 'SANS H2O']):
    data[code] = np.loadtxt(code + '.dat', comments = comments)
    plt.scatter(data[code][:,0], data[code][:,1], label=label)
    
profiles = ['PepsiSAXS.csv', 'deut0_d2o0_exch0_PepsiSANS.csv', 'deut0_d2o100_exch0_PepsiSANS.csv', 'FoXS.csv', ]
labels = ['PepsiSAXS','PepsiSANS-H2O','PepsiSANS-D2O', 'FoXS',]

for p, label in zip(profiles, labels):
    p = system + '_' + p
    calc = np.genfromtxt(p, delimiter=",", skip_header=1)
    plt.plot(calc[:,1], calc[:,2]/calc[0,2], label=label)

plt.xlabel('Q/$\AA^{-1}$')
plt.ylabel('(IQ)')
plt.yscale('log')
plt.xlim((0,0.5))
plt.ylim((0.001,1.1))
plt.title(system)
plt.legend()
plt.show()

# Generating different conformations

As the proteins are flexible, the single crystal reference structure is not enough to model the different conformations that it can adopt in a solvent. Those can be generated using either RRT (Rapidly-exploring Random Trees) or NOLB (NOn-Linear rigid Block Normal Model Analysis). 

The RRT modeling is computationally more expensive and requires a good knowledge of the system, as the user must provide a file specifying the flexible residues, so here we will use the second method.

To perform NOLB modeling, you need to provide a `pdb_file` and specify the following options:

- **num_iter**: This parameter represents the number of iterations for the NOLB algorithm. It determines how many iterations the algorithm will perform to generate the models. Increasing the number of iterations can lead to a more refined sampling of conformational space.

- **num_modes**: This parameter specifies the number of modes (or models) to be generated. Each mode represents a distinct conformation.

After executing the command, the NOLB algorithm will generate the specified number of modes. You can examine the created nodes (modes) in the output.

### Irina
How can the `flex_file` needed by RRT be generated (semi-)automatically or as easily as possible?

Useful to give names of generated files?

In [ ]:
pdb_file = system + '.pdb'
modelpdb_nolb(pdb_file, num_iter=ITERATIONS, num_modes=MODES)

# Fit each of the conformations to the experimental SAXS data

This is done using `fitsaxs` and either `foxs` or `pepsisaxs`.

If you choose `core="foxs"`, the following options are taken into account:

```python
fitsaxs(pdb_files, data_file, core="foxs", c1_low=0.99, c1_up=1.05, c2_low=-2, c2_up=4, bg=0, 
        hyd=False)
```
- **c1** is the scaling of the atomic radius, which controls the excluded volume of the molecule. The default value is c1 = 1.0. During fitting, a range of values is allowed, with a 1% decrease and up to a 5% increase in the radius (0.99 ≤ c1 ≤ 1.05). The **c1_low** and **c1_up** parameters define this range.
- **c2** is used to adjust the difference between the densities of the hydration layer and the bulk water. It controls the density of the water layer around the molecule. The default value is c2 = 0.0. The value of c2 can vary from 0 to 4.0, representing up to four water molecule neighbors for an exposed solute atom. Negative values are also allowed (-2.0 ≤ c2 ≤ 4.0) to account for a lower hydration shell density. The **c2_low** and **c2_up** parameters define this range.
- **bg** is an option for background adjustment, which is not used by default.
- **hyd** is a boolean flag that indicates whether to explicitly consider hydrogens in the PDB files. The default value is False. If you want to use hydrogens, set `hyd=True`, assuming that all hydrogen atoms are listed in the PDB file.

For `core="pepsisaxs"`, the available options are:

```python
fitsaxs(pdb_files, data_file, core="pepsisaxs", bg=0, hyd=False, scale=1, int_0=1, neg=False,
        no_smear=False, hyd_shell=5, conc=1, abs_int=0, bulk_SLD=1e-5)
```
- **bg** is an option for background adjustment, which is not used by default.
- **hyd** is a boolean flag that indicates whether to explicitly consider hydrogens in the PDB files.
- **scale** is a scaling factor between the experimental intensity $I_{exp}$ and the theoretical intensity $I_{theory}$.
- **int_0** sets $I(0)$ to a constant value.
- **neg** is a flag that allows for a negative contrast of the hydration shell upon fitting.
- **no_smear** disables the data smearing during fitting.
- **hyd_shell** represents the hydration shell contrast as a percentage of the bulk value. The default is 5%. If this parameter is omitted, the contrast will be adjusted automatically during fitting.
- **conc** specifies the sample concentration in mg/mL. The default is 1 mg/mL. This parameter is only used when the `abs_int` option is enabled.
- **abs_int** enables the fitting of absolute intensity, in +-%.
- **bulk_SLD** allows for the explicit specification of the bulk SLD (Scattering Length Density) if different from water.

### Irina
Why using FoXS gives the same chi^2 for all the pdb files?

Why the fits stop at 0.5 AA-1 and do not cover the same range as the exp data?

PepsiSAXS gives different chi^2 for each file, but with values rather larger than FoXS? Why?

Fitting with PepsiSANS the protein in D2O: Exp curves seem to have an extra background? Why is this not fitted? Or is it not a real background and cannot be fitted? In any case, simulation and experiment do not really match, why?

It would be nice to provide directly in the notebook a summary of the main fitted parameters, as well as the names of the generated output files (e.g. *output.txt and .log files for pepsisaxs). 

In [ ]:
fitsaxs([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][0] + '.dat', core="foxs")

Using PepsiSAXS:

In [ ]:
fitsaxs([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][0] + '.dat', core="pepsisaxs")

# Fit each of the conformations to the experimental SANS data

To fit the SANS data we use `fitsans`. The syntax is similar to that of `fitsaxs`, but now we need to indicate the H/D level of the protein (keyword `deut_level`) and the buffer (keyword `d2o_level`). It is also possible to define a few other options:

```python
fitsans(pdb_files, data_file, deut_level=[0], d2o_level=[0], exchange=[0], bg=0, hyd=False, scale=1,
        neg=False, no_smear=False, hyd_shell=5, conc=1, abs_int=0, bulk_SLD=1e-5)
```

- **deut_level** stands for Molecule deuteration. It represents the level of deuteration in the molecules under investigation. Deuteration refers to the replacement of hydrogen atoms with deuterium atoms. The **deut_level** value should be in the range from 0 to 1.
- **d2o_level** stands for Buffer deuteration. It represents the level of deuteration in the solvent or buffer used in the experiment. The **d2o_level** value should be in the range from 0 to 1.
- **exchange** stands for Exchange rate. It represents the rate at which hydrogen atoms in the molecule exchange with deuterium atoms. Higher exchange rates indicate faster exchange between hydrogen and deuterium atoms. The **exchange** value should be in the range from 0 to 1.
- **bg** is an option for background adjustment, which is not used by default.
- **hyd** is a boolean flag that indicates whether to explicitly consider hydrogens in the PDB files.
- **scale** is a scaling factor between the experimental intensity $I_{exp}$ and the theoretical intensity $I_{theory}$.
- **neg** is a flag that allows for a negative contrast of the hydration shell upon fitting.
- **no_smear** disables the data smearing during fitting.
- **hyd_shell** represents the hydration shell contrast as a percentage of the bulk value. The default is 5%. If this parameter is omitted, the contrast will be adjusted automatically during fitting.
- **conc** specifies the sample concentration in mg/mL. The default is 1 mg/mL. This parameter is only used when the `abs_int` option is enabled.
- **abs_int** enables the fitting of absolute intensity, in +-%.
- **bulk_SLD** allows for the explicit specification of the bulk SLD (Scattering Length Density) if different from water.

We start by fitting the data measured in D2O, so we set *d2o_level = 1*.

In [ ]:
fitsans([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][1] + '.dat', 
        deut_level=0, d2o_level=1, exchange=0, bg=1)

And now we fit the SANS data taken on H2O, by setting *d2o_level = 0*.

### Attention:

SANS data taken on H2O have worse statistics, so some of the files in SASBDB contain some "bad" points with negative errors. These points will generate an error when trying to plot the graph. In this case, edit the file in the server, remove these points and call fitsans again.

### Irina
Add handling of points with negative intensity/error bar into fitsans.

In [ ]:
fitsans([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][2] + '.dat', 
        deut_level=0, d2o_level=0, exchange=0, bg=0)

# Multi-State Modeling with `multimodelfit`

In data analysis, dealing with heterogeneous samples is common, where heterogeneity can be both in composition and conformation. For interpreting data collected from such samples, a multi-state model is essential. A multi-state model involves multiple co-existing structural states and parameters, including weights for each state. This is done by `multimodelfit`, which can be used with either SAXS or SANS data. 

```python
multimodelfit(pdb_files, data_file, type="saxs", ensemble_size=10, bestK=1000, chi_perc=0.3, chi=0,
              min_weight=0.05, max_q=0.5, c1_low=0.99, c1_up=1.05, c2_low=-0.5, c2_up=2, multimodel=1,
              bg=0, nnls=False,
              deut_level=[0], d2o_level=[0], exchange=[0], conc=1, abs_int=0, hyd=False, 
              bulk_SLD=1e-5, no_smear=False, scale=1, neg=False, hyd_shell=5):
```    

For both SAXS and SANS multi-state modeling, the following parameters are common:

- **pdb_files**: A collection of PDB files for multi-model fitting, with a minimum of 2 files.
- **data_file**: The data file to fit, typically in .dat format.
- **ensemble_size**: The maximum ensemble size, with a default of 10.
- **bestK**: Default value is 1000.
- **chi_perc**: The chi value percentage threshold for profile similarity, defaulting to 0.3.
- **chi**: A chi-based threshold, defaulting to 0.
- **min_weight**: The minimum weight threshold for a profile to contribute to the ensemble, defaulting to 0.05.
- **max_q**: The maximum q value, with a default of 0.5.
- **c1_low**: The minimum c1 value, defaulting to 0.99.
- **c1_up**: The maximum c1 value, defaulting to 1.05.
- **c2_low**: The minimum c2 value, defaulting to -0.5.
- **c2_up**: The maximum c2 value, defaulting to 2.
- **multimodel**: Option to read models, with choices 1, 2, or 3. 1: read the first Model only (default); 2: read each model into a separate structure; 3: read all models into a single structure;
- **bg**: Background adjustment option, not used by default.
- **nnls**: Running Non-negative Least Square on all profiles, defaulting to False.

When using `multimodelfit` on SANS data, the following options are also available:

- **deut_level** stands for Molecule deuteration. It represents the level of deuteration in the molecules under investigation. Deuteration refers to the replacement of hydrogen atoms with deuterium atoms. The **deut_level** value should be in the range from 0 to 1.
- **d2o_level** stands for Buffer deuteration. It represents the level of deuteration in the solvent or buffer used in the experiment. The **d2o_level** value should be in the range from 0 to 1.
- **exchange** stands for Exchange rate. It represents the rate at which hydrogen atoms in the molecule exchange with deuterium atoms. Higher exchange rates indicate faster exchange between hydrogen and deuterium atoms. The **exchange** value should be in the range from 0 to 1.
- **conc** specifies the sample concentration in mg/mL. The default is 1 mg/mL. This parameter is only used when the `abs_int` option is enabled.
- **abs_int** enables the fitting of absolute intensity, in +-%.
- **hyd** is a boolean flag that indicates whether to explicitly consider hydrogens in the PDB files.
- **bulk_SLD** allows for the explicit specification of the bulk SLD (Scattering Length Density) if different from water.
- **no_smear** disables the data smearing during fitting.
- **scale** is a scaling factor between the experimental intensity $I_{exp}$ and the theoretical intensity $I_{theory}$.
- **neg** is a flag that allows for a negative contrast of the hydration shell upon fitting.
- **hyd_shell** represents the hydration shell contrast as a percentage of the bulk value. The default is 5%. If this parameter is omitted, the contrast will be adjusted automatically during fitting.

### Irina
Why all the multifits use only 1 PDB?

In [ ]:
# Multifit using the SAXS data
mf_saxs = multimodelfit([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][0] + '.dat')

In [ ]:
#Multifit using the SANS data on D2O
mf_sans_d2o = multimodelfit([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][1] + '.dat', 
                            type='sans', d2o_level=1)

In [ ]:
#Multifit using the SANS data on D2O
mf_sans_h2o = multimodelfit([system + '_nlb_{}.pdb'.format(i+1) for i in range(MODES)], codes[system][1][2] + '.dat', 
                            type='sans', d2o_level=0)

# Multifit with prion example

In [ ]:
modes = modelpdb_flex("prion.pdb", "prion_linkers.txt", models="sep")

In [ ]:
mmf = multimodelfit(["nodes{}.pdb".format(i) for i in range(1,101)], "prion_iq.dat")

In [ ]:
# Then we can have a look on the contributing PDB file(s)
string = mmf["Contributing PDB file(s)"][0] #the index is the same as in the table above
file_list = [file.strip() for file in string.split(',')]
interact(show_pdb, pdb_file=file_list)